In [1]:
import pickle
import math
import numpy as np
import time
from collections import defaultdict

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

In [2]:
AutoML_path = ""
log_folder = "tpot_log"

log_DDN = AutoML_path+log_folder+'/DDN_single_time'
log_pipeline_steps = AutoML_path+log_folder+'/pipeline_steps_class_02272022_tpot'

combination_name = 'DDN+OneHot'
log_target_test_pred_time = AutoML_path+log_folder+'/'+combination_name+'_target_test_pred_time_03102022'


In [3]:
DDN = pickle.load(open(log_DDN, "rb"))
print(len(DDN))

pipeline_steps_class = pickle.load(open(log_pipeline_steps, 'rb'))
print(len(pipeline_steps_class))

pipeline_steps_class = [i for i in pipeline_steps_class if i[3] != "Timeout" and not np.isinf(i[3])]
print(len(pipeline_steps_class))

39
12863
11797


In [4]:
target_test_pred_time = {}
data_names = list(DDN.keys())
data_embed = DDN

for data_index, data_name in enumerate(data_names):
    print("="*40)
    print(data_index, data_name)
    
    task_test = [data_name]
    candidate_score = DDN[data_name]
    prev_key = None
    dict_key = None
    len_task_train = 0
    prev_len_task_train = 0
    for threshold in np.arange(0.05, 0.5, 0.05):
        prev_key = dict_key        
        dict_key = '_'.join([data_name, combination_name, str(round(threshold,2))])
        
        start = time.time()
        task_train = [i for i in data_names if i!=data_name and abs(DDN[i]-candidate_score)<threshold]
        prev_len_task_train = len_task_train
        len_task_train = len(task_train)
        if len(task_train) == 0:
            target_test_pred_time[dict_key] = 'N.A.'
            pickle.dump(target_test_pred_time, open(log_target_test_pred_time, "wb"))
            continue
            
        if len_task_train == prev_len_task_train:
            print("+"*30)
            print(prev_key, dict_key)
            print(prev_len_task_train, len_task_train)
            target_test_pred_time[dict_key] =  target_test_pred_time[prev_key]
            continue

        X_train = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_train and i[3] != "Timeout" and not np.isinf(i[3])])
        y_train = np.array([i[3] for i in pipeline_steps_class if i[0] in task_train and i[3] != "Timeout" and not np.isinf(i[3])])
        X_test = np.array([np.hstack((data_embed[i[0]], i[1])) for i in pipeline_steps_class if i[0] in task_test and i[3] != "Timeout" and not np.isinf(i[3])])
        y_test = np.array([i[3] for i in pipeline_steps_class if i[0] in task_test and i[3] != "Timeout" and not np.isinf(i[3])])
        print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)
        print("X_test shape:", X_test.shape, "y_test shape:", y_test.shape)

        model = MLPRegressor(hidden_layer_sizes=(256,128,64), learning_rate='adaptive', random_state=19)
        model.fit(X_train, y_train)
        end = time.time()
        duration = end-start

        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        print("Threshold", threshold, "MAE:", mae, "MSE:", mse)
        print("Time:", duration)
        target_test_pred_time[dict_key] = (y_test, y_pred, duration, mae, mse)
        pickle.dump(target_test_pred_time, open(log_target_test_pred_time, 'wb'))
    


0 adult
X_train shape: (2129, 96) y_train shape: (2129,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.05 MAE: 0.05189020743695941 MSE: 0.005647338782063457
Time: 0.8659389019012451
X_train shape: (4352, 96) y_train shape: (4352,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.1 MAE: 0.042083116901707324 MSE: 0.004234682220712752
Time: 2.1822471618652344
X_train shape: (7632, 96) y_train shape: (7632,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.15000000000000002 MAE: 0.042355171221876355 MSE: 0.0040692687556112455
Time: 3.867539882659912
X_train shape: (8814, 96) y_train shape: (8814,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.2 MAE: 0.04558723551639133 MSE: 0.005015556322323171
Time: 4.394630193710327
X_train shape: (10228, 96) y_train shape: (10228,)
X_test shape: (271, 96) y_test shape: (271,)
Threshold 0.25 MAE: 0.0440478571600567 MSE: 0.005360416621071801
Time: 6.273854970932007
X_train shape: (10830, 96) y_train shape: (10830,)
X_

Threshold 0.05 MAE: 0.052099775533244776 MSE: 0.005057908975640966
Time: 1.2363600730895996
X_train shape: (4379, 96) y_train shape: (4379,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.1 MAE: 0.06395687517758306 MSE: 0.005990413494821717
Time: 2.0589919090270996
X_train shape: (8336, 96) y_train shape: (8336,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.15000000000000002 MAE: 0.04861968739429324 MSE: 0.005289157119528846
Time: 3.8042280673980713
X_train shape: (8771, 96) y_train shape: (8771,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.2 MAE: 0.05232264824864676 MSE: 0.005191127166561439
Time: 3.7175862789154053
X_train shape: (10185, 96) y_train shape: (10185,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.25 MAE: 0.04678377067988808 MSE: 0.004578114587925511
Time: 4.556777238845825
X_train shape: (10787, 96) y_train shape: (10787,)
X_test shape: (314, 96) y_test shape: (314,)
Threshold 0.3 MAE: 0.04977471412640685 MSE: 0.005108300313

X_train shape: (6402, 96) y_train shape: (6402,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.1 MAE: 0.04689387814631856 MSE: 0.004732275187052941
Time: 4.297590970993042
++++++++++++++++++++++++++++++
cnae-9_DDN+OneHot_0.1 cnae-9_DDN+OneHot_0.15
20 20
X_train shape: (7661, 96) y_train shape: (7661,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.2 MAE: 0.046542679414429466 MSE: 0.004630024952994455
Time: 3.4750421047210693
X_train shape: (8797, 96) y_train shape: (8797,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.25 MAE: 0.06972226908750574 MSE: 0.008721001295119527
Time: 4.022511005401611
X_train shape: (9926, 96) y_train shape: (9926,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.3 MAE: 0.0503579155531683 MSE: 0.004616600096426373
Time: 3.656125068664551
X_train shape: (10492, 96) y_train shape: (10492,)
X_test shape: (288, 96) y_test shape: (288,)
Threshold 0.35000000000000003 MAE: 0.04472458740833878 MSE: 0.003733081143883238
Time: 4

X_train shape: (3647, 96) y_train shape: (3647,)
X_test shape: (162, 96) y_test shape: (162,)
Threshold 0.1 MAE: 0.2563054698248244 MSE: 0.10423382816027607
Time: 2.471853017807007
X_train shape: (5376, 96) y_train shape: (5376,)
X_test shape: (162, 96) y_test shape: (162,)
Threshold 0.15000000000000002 MAE: 0.24931647003869242 MSE: 0.09943394366117246
Time: 2.492314100265503
X_train shape: (7311, 96) y_train shape: (7311,)
X_test shape: (162, 96) y_test shape: (162,)
Threshold 0.2 MAE: 0.2544056487593479 MSE: 0.1036730852835413
Time: 2.9484469890594482
X_train shape: (8969, 96) y_train shape: (8969,)
X_test shape: (162, 96) y_test shape: (162,)
Threshold 0.25 MAE: 0.24789663651155058 MSE: 0.0999530142653531
Time: 3.516803026199341
X_train shape: (10920, 96) y_train shape: (10920,)
X_test shape: (162, 96) y_test shape: (162,)
Threshold 0.3 MAE: 0.25476819761555125 MSE: 0.10541850895196567
Time: 4.6160173416137695
X_train shape: (11401, 96) y_train shape: (11401,)
X_test shape: (162, 96

Threshold 0.25 MAE: 0.036834819162030075 MSE: 0.002295156301200217
Time: 3.1125690937042236
X_train shape: (7675, 96) y_train shape: (7675,)
X_test shape: (285, 96) y_test shape: (285,)
Threshold 0.3 MAE: 0.04722889591874588 MSE: 0.0034709789574701847
Time: 3.482707977294922
X_train shape: (9515, 96) y_train shape: (9515,)
X_test shape: (285, 96) y_test shape: (285,)
Threshold 0.35000000000000003 MAE: 0.04293303106517921 MSE: 0.0028202327953251965
Time: 3.680206775665283
X_train shape: (11512, 96) y_train shape: (11512,)
X_test shape: (285, 96) y_test shape: (285,)
Threshold 0.4 MAE: 0.051834068246471 MSE: 0.00392982927173789
Time: 4.034125089645386
++++++++++++++++++++++++++++++
higgs_DDN+OneHot_0.4 higgs_DDN+OneHot_0.45
38 38
21 jannis
X_train shape: (1112, 96) y_train shape: (1112,)
X_test shape: (302, 96) y_test shape: (302,)
Threshold 0.05 MAE: 0.046752841183508156 MSE: 0.0034743928480810583
Time: 0.6309120655059814
X_train shape: (2850, 96) y_train shape: (2850,)
X_test shape: (3

X_train shape: (10088, 96) y_train shape: (10088,)
X_test shape: (411, 96) y_test shape: (411,)
Threshold 0.4 MAE: 0.07808203161784254 MSE: 0.03773829168867549
Time: 3.923114061355591
X_train shape: (10690, 96) y_train shape: (10690,)
X_test shape: (411, 96) y_test shape: (411,)
Threshold 0.45 MAE: 0.07344418687373325 MSE: 0.033356204003149884
Time: 3.854952096939087
26 kr-vs-kp
X_train shape: (1635, 96) y_train shape: (1635,)
X_test shape: (362, 96) y_test shape: (362,)
Threshold 0.05 MAE: 0.03706861850703529 MSE: 0.0035562625379382895
Time: 0.9651188850402832
X_train shape: (3609, 96) y_train shape: (3609,)
X_test shape: (362, 96) y_test shape: (362,)
Threshold 0.1 MAE: 0.043642862899474014 MSE: 0.0036511993291265107
Time: 1.388969898223877
X_train shape: (5201, 96) y_train shape: (5201,)
X_test shape: (362, 96) y_test shape: (362,)
Threshold 0.15000000000000002 MAE: 0.03412979064375015 MSE: 0.002963196826723261
Time: 3.855408191680908
X_train shape: (6328, 96) y_train shape: (6328,)

X_train shape: (7483, 96) y_train shape: (7483,)
X_test shape: (343, 96) y_test shape: (343,)
Threshold 0.4 MAE: 0.07633145836637924 MSE: 0.0083464978224888
Time: 3.0529091358184814
X_train shape: (9457, 96) y_train shape: (9457,)
X_test shape: (343, 96) y_test shape: (343,)
Threshold 0.45 MAE: 0.06063533075721263 MSE: 0.005720227922955146
Time: 3.625488042831421
31 phoneme
X_train shape: (2074, 96) y_train shape: (2074,)
X_test shape: (326, 96) y_test shape: (326,)
Threshold 0.05 MAE: 0.04842067642911141 MSE: 0.0038252455428321526
Time: 1.365583896636963
X_train shape: (4032, 96) y_train shape: (4032,)
X_test shape: (326, 96) y_test shape: (326,)
Threshold 0.1 MAE: 0.04595163855389385 MSE: 0.003205339728016105
Time: 3.051762819290161
X_train shape: (7298, 96) y_train shape: (7298,)
X_test shape: (326, 96) y_test shape: (326,)
Threshold 0.15000000000000002 MAE: 0.05641146069569714 MSE: 0.004520710991531173
Time: 5.207041025161743
X_train shape: (8990, 96) y_train shape: (8990,)
X_test 

X_train shape: (10692, 96) y_train shape: (10692,)
X_test shape: (409, 96) y_test shape: (409,)
Threshold 0.45 MAE: 0.04633667199265559 MSE: 0.004022440079788102
Time: 5.042578935623169
36 sylvine
X_train shape: (1925, 96) y_train shape: (1925,)
X_test shape: (368, 96) y_test shape: (368,)
Threshold 0.05 MAE: 0.05848789705953771 MSE: 0.007893879932409231
Time: 0.8184449672698975
X_train shape: (6322, 96) y_train shape: (6322,)
X_test shape: (368, 96) y_test shape: (368,)
Threshold 0.1 MAE: 0.062351895583030725 MSE: 0.00756694319326004
Time: 3.1754560470581055
++++++++++++++++++++++++++++++
sylvine_DDN+OneHot_0.1 sylvine_DDN+OneHot_0.15
20 20
X_train shape: (7903, 96) y_train shape: (7903,)
X_test shape: (368, 96) y_test shape: (368,)
Threshold 0.2 MAE: 0.05983569720020481 MSE: 0.008059435491115503
Time: 3.063828945159912
X_train shape: (8717, 96) y_train shape: (8717,)
X_test shape: (368, 96) y_test shape: (368,)
Threshold 0.25 MAE: 0.056908930053069975 MSE: 0.007523184254917082
Time: 